<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/4_rag_recursive_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia
    * Usar segmentação por similaridade
    * Busca densa
    * Recursive Retriver

# Instalando pacotes

In [12]:
!pip install -q llama-index llama-index-llms-groq sentence-transformers llama-index-embeddings-huggingface transformers

# Importando pacotes

In [13]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [14]:
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.schema import TextNode, IndexNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import FaithfulnessEvaluator, CorrectnessEvaluator, RelevancyEvaluator
from llama_index.core.response.notebook_utils import display_source_node, display_response
import json
import pandas as pd
from rich.pretty import pprint
from tqdm import tqdm
from IPython.display import Markdown, display

import nest_asyncio
nest_asyncio.apply()


# GitHub

In [15]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

fatal: destination path 'ia024-projeto-rag' already exists and is not an empty directory.


In [16]:
DATA_DIR = '/content/ia024-projeto-rag'

In [17]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [18]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [19]:
update_local_repo()

/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.


# RAG - Recursive Retriever

## Preparando dados

In [20]:
embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

Settings.embed_model = embed_model
Settings.llm         = Groq(model="llama3-70b-8192")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
import glob

def get_json_files(directory):
    return glob.glob(os.path.join(directory, "*.json"))


base_nodes     = []

print(f"Carregando Estatuto e Organizações Didáticas...")
directory_path = f"{DATA_DIR}/2-dataset/"
json_files     = get_json_files(directory_path)

for json_file in json_files:
    print(f"Carregando: {json_file}")
    with open(json_file, 'r') as f:
        documento = json.load(f)

    documento_titulo = documento['titulo']
    documento_texto  = documento['texto_compacto']

    for parte in documento['partes']:
        metadata = {}
        metadata["documento"] = documento_titulo

        titulo_nome    = parte['titulo']
        titulo_artigos = parte['artigos']
        capitulos      = parte['capitulos']

        metadata["titulo"] = titulo_nome
        if titulo_artigos != "":
            node = TextNode(text=titulo_artigos, metadata=metadata)
            base_nodes.append(node)

        for capitulo in capitulos:
            cap_titulo  = capitulo['titulo']
            cap_artigos = capitulo['artigos']

            metadata["capitulo"] = cap_titulo
            node = TextNode(text=cap_artigos, metadata=metadata)
            base_nodes.append(node)
    print(f"{len(base_nodes)} nodes carregados")

print(f"Carregando portarias: {directory_path}")
directory_path = f"{DATA_DIR}/2-dataset/2-boituva/portarias"
json_files     = get_json_files(directory_path)

for json_file in json_files:
    print(f"Carregando: {json_file}")
    with open(json_file, 'r') as f:
        documento = json.load(f)

    for portaria in documento:

        portaria_titulo = portaria['title:']
        portaria_texto  = portaria['texto_completo']

        metadata = {}
        metadata["campus"]    = portaria['campus']
        metadata["documento"] = portaria_titulo
        node = TextNode(text=portaria_texto, metadata=metadata)
        base_nodes.append(node)

    print(f"{len(base_nodes)} nodes carregados")

for i, node in enumerate(base_nodes):
    node.id_ = f"node_{i}"

print(f"Total de nodes carregados: {len(base_nodes)}")

Carregando Estatuto e Organizações Didáticas...
Carregando: /content/ia024-projeto-rag/2-dataset/estatuto.json
17 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/organização_didática_da_educação_básica.json
47 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/organização_didática_dos_cursos_de_graduação.json
76 nodes carregados
Carregando portarias: /content/ia024-projeto-rag/2-dataset/
Carregando: /content/ia024-projeto-rag/2-dataset/2-boituva/portarias/boituva-2015.json
78 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/2-boituva/portarias/boituva-2023.json
145 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/2-boituva/portarias/boituva-2020.json
162 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/2-boituva/portarias/boituva-2021.json
248 nodes carregados
Carregando: /content/ia024-projeto-rag/2-dataset/2-boituva/portarias/boituva-2018.json
396 nodes carregados
Carregando: /content/ia024-projeto-ra

In [22]:
pprint(base_nodes[-5:])

[
│   TextNode(
│   │   id_='node_448',
│   │   embedding=None,
│   │   metadata={'campus': 'boituva', 'documento': 'Portaria nº 3.903, de 04 de novembro de 2015'},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='MINISTÉRIO DA EDUCAÇÃO. INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO. CAMPUS BOITUVA. PORTARIA Nº 33/2024 - DRG/BTV/IFSP DE 2 DE MAIO DE 2024 O DIRETOR GERAL DO CAMPUS BOITUVA, DO INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO, no uso de suas atribuições legais delegadas pela Portaria nº 3.903, de 04 de novembro de 2015, RESOLVE: Art 1.º - DESIGNAR, o servidor Rafael Augusto Rocha Maia; CPF: XXX.444.188-XX como titular e, como substituto o servidor Helio Falci Junior; CPF: XXX.991.188-XX para nos termos do artigo 117 da Lei nº 14.133/2021, exercer a fiscalização e acompanhamento do contrato abaixo relacionado:. |Contrato n°|Empresa|Objeto| |---|---|---| |02710/2024|Ragnar Segurança Ltda,|Prestação de serviço continuado de vigilância patrimonial desarmada;|. Art 2.º - ATRIBUIÇÕES DO FISCAL: acompanhar e fiscalizar o fiel cumprimento do Contrato de cláusulas avençadas, opinar sobre a oportunidade e conveniência de prorrogação de vigência ou aditamento de objeto, bem como comunicar as irregularidades eventualmente constatadas à Coordenadoria de Contratos e Convênios. Art 3º - Esta Portaria tem vigência até 01/05/2025. Art 4º - Esta Portaria entra em vigor na data de sua publicação. De ciência. Publique-se. Documento assinado eletronicamente. FELIPE AUGUSTO FERREIRA DE ALMEIDA Publicado no site institucional em 02.05.2024. Documento assinado eletronicamente por:. Felipe Augusto Ferreira de Almeida Felipe Augusto Ferreira de Almeida, DIRETOR(A) GERAL - CD2 - DRG/BTV, em 02/05/2024 18:19:00. Este documento foi emitido pelo SUAP em 02/05/2024. Para comprovar sua autenticidade, faça a leitura do QRCode ao lado ou acesse https://suap.ifsp.edu.br/autenticar-documento/ e forneça os dados abaixo:. |Código Verificador:|741820| |---|---| |Código de Autenticação:|f347eaa4df|. PORTARIA Nº 33/2024 - DRG/BTV/IFSP. PORTARIA Nº 33/2024 - DRG/BTV/IFSP',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   TextNode(
│   │   id_='node_449',
│   │   embedding=None,
│   │   metadata={'campus': 'boituva', 'documento': 'Portaria nº 3.903, de 04 de novembro de 2015'},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='MINISTÉRIO DA EDUCAÇÃO. INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO. CAMPUS BOITUVA. PORTARIA Nº 1/2024 - DRG/BTV/IFSP DE 5 DE JANEIRO DE 2024 O DIRETOR-GERAL EM EXERCÍCIO DO CAMPUS BOITUVA, DO INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO, no uso de suas atribuições legais delegadas pela Portaria nº 3.903, de 04 de novembro de 2015 e o que diz a PORTARIA Nº 6583/IFSP, de 20 de dezembro de 2021, RESOLVE: Art 1.º - INSTITUIR, a Comissão de Matrícula do Campus Boituva (CM), para auxílio a Coordenadoria de Registros Acadêmicos nos períodos de matrícula dos processos seletivos ofertados, para ingressantes no primeiro semestre de 2024. Art 2.º - DESIGNAR, os servidores abaixo relacionados para, sob a Presidência do primeiro, compor a comissão supracitada. Mariane de Almeida Silva Lara Felipe Gobo Bruno Lívia Pereira de Paula Mariangela Alves Dias Fernando Maffeis Leumas Castelli Miriam Cristina Peruzzi Soares Belote Bruno Fortunato Lima Patrícia da Costa Oliveira Art 3º - Esta Portaria tem vigência até o dia 30 de abril de 2024. Art 4º - Esta Portaria entra em vigor na data de sua publicação. De ciência. Publique-se. Documento assinado eletronicamente. FELIPE GOBO BRUNO Publicado no site institucional em 05.01.2024. Documento assinado eletronicamente 

## Construindo Retriever e Query Engine

In [23]:
# Set Up Recursive Retriever
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes]

all_nodes = []
for base_node in base_nodes:

    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes]
        all_nodes.extend(sub_inodes)

    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

all_nodes_dict = {n.node_id: n for n in all_nodes}

print(f"Total de nodes: {len(all_nodes)}")

Metadata length (85) is close to chunk size (128). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (80) is close to chunk size (128). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (95) is close to chunk size (128). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (98) is close to chunk size (128). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Total de nodes: 7912


In [24]:
pprint(all_nodes[-2])

IndexNode(
│   id_='321efebd-be72-4643-bf06-140dcc08549f',
│   embedding=None,
│   metadata={'campus': 'boituva', 'documento': 'Portaria nº 3.903, de 04 de novembro de 2015'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='node_452',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={'campus': 'boituva', 'documento': 'Portaria nº 3.903, de 04 de novembro de 2015'},
│   │   │   hash='067a3b53bb3a814315b3a42bd06bc2dfb673e043a3be452cb468c9ff00b180b2'
│   │   ),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   node_id='f9af0eea-6ab1-40b6-a867-a8c87e6698d1',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={'campus': 'boituva', 'documento': 'Portaria nº 3.903, de 04 de novembro de 2015'},
│   │   │   hash='40d49a4da1da31ae95b463b40ce1fe24a4a357d27166363766705aa8f8aeacf7'
│   │   )
│   },
│   text='De ciência. Publique-se. Documento assinado eletronicamente. FELIPE GOBO BRUNO Publicado no site institucional em 08.01.2024. Documento assinado eletronicamente por:. Felipe Gobo Bruno Felipe Gobo Bruno, COORDENADOR(A) - FG2 - CDI-BTVCOORDENADOR(A) - FG2 - CDI-BTV, em 08/01/2024 16:48:11. Este documento foi emitido pelo SUAP em 08/01/2024. Para comprovar sua autenticidade, faça a leitura do QRCode ao lado ou acesse https://suap.ifsp.edu.br/autenticar-documento/ e forneça os dados abaixo:. |Código Verificador:|677015| |---|---| |Código de Autenticação:|838e98bf5f|. PORTARIA Nº 2/2024 - DRG/BTV/IFSP. PORTARIA Nº 2/2024 - DRG/BTV/IFSP',
│   mimetype='text/plain',
│   start_char_idx=1317,
│   end_char_idx=1956,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n',
│   index_id='node_452',
│   obj=None
)

In [25]:
from llama_index.core import StorageContext, load_index_from_storage

index_dir = f"{DATA_DIR}/4-vector_index"

# Carrega o indice existente ou cria um novo
if os.path.exists(index_dir):

    storage_context = StorageContext.from_defaults(persist_dir=index_dir)

    # load index
    index = load_index_from_storage(storage_context)
else:

    index = VectorStoreIndex(all_nodes)
    index.storage_context.persist(index_dir)


In [26]:
import shutil

# Caminho da pasta que você deseja compactar
pasta_para_compactar = f"{DATA_DIR}/4-vector_index"

# Caminho e nome do arquivo zip de saída (sem a extensão .zip)
arquivo_zip_saida = f"{DATA_DIR}/4-vector_index"

# Cria o arquivo zip
shutil.make_archive(arquivo_zip_saida, 'zip', pasta_para_compactar)

'/content/ia024-projeto-rag/4-vector_index.zip'

In [27]:
save_on_git("Salvando o índice de vetores")

/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [28]:
vector_retriever = index.as_retriever(similarity_top_k=20)

retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=all_nodes_dict,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    verbose=True
)

# Implementando o Rerank

In [29]:
!pip install -q rerankers

In [30]:
from rerankers import Reranker
import torch
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from llama_index.core.query_engine import RetrieverQueryEngine
from IPython.display import Markdown, display
import pandas as pd
import gc

In [31]:
def rerank(query, nodes, top_k=5):
    docs = [node.get_content() for node in nodes]

    ranker = Reranker(
        "unicamp-dl/monoptt5-large",
        inputs_template="Pergunta: {query} Documento: {text} Relevante:",
        dtype=torch.float32
    )

    results = ranker.rank(query, docs)

    scored_nodes = list(zip(nodes, [result.score for result in results]))
    scored_nodes.sort(key=lambda x: x[1], reverse=True)

    top_nodes = [node for node, score in scored_nodes[:top_k]]

    # Free up memory
    del ranker
    torch.cuda.empty_cache()
    gc.collect()

    return top_nodes

In [32]:
class RerankedRetrieverQueryEngine(RetrieverQueryEngine):
    def __init__(self, retriever, reranker, top_k=5, **kwargs):
        super().__init__(retriever=retriever, **kwargs)
        self.reranker = reranker
        self.top_k = top_k

    def query(self, query_str):
        response = super().query(query_str)
        nodes_to_rerank = response.source_nodes
        top_reranked_nodes = self.reranker(query_str, nodes_to_rerank, self.top_k)
        response.source_nodes = top_reranked_nodes
        return response

In [33]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [34]:
def get_contexts(response):
    contexts = []
    for node in response.source_nodes:
        metadata = ', '.join(node.metadata.values())
        context  = node.get_content()
        contexts.append(f"Fonte: {metadata} # Contextos: {context}")
    return contexts

In [35]:
def parse_retry_after(error_message):
    try:
        error_data = json.loads(error_message)
        return error_data['error']['message'].split('Please try again in ')[1].split('s.')[0]
    except Exception as e:
        print(f"Failed to parse retry-after duration: {e}")
        return None

In [36]:
index_dir = f"{DATA_DIR}/4-vector_index"

if os.path.exists(index_dir):
    storage_context = StorageContext.from_defaults(persist_dir=index_dir)
    index = load_index_from_storage(storage_context)
else:
    index = VectorStoreIndex(all_nodes)
    index.storage_context.persist(index_dir)

vector_retriever = index.as_retriever(similarity_top_k=20)

retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=all_nodes_dict,
)

In [37]:
query_engine_with_rerank = RerankedRetrieverQueryEngine(
    retriever=retriever,
    reranker=rerank,
    top_k=5
)

In [38]:
def evaluate_pair(pair, max_retries=5):
    result = {}
    query = pair['question']
    ground_truth = pair['answer']

    result['question'] = query
    result['ground_truth'] = ground_truth

    retries = 0
    while retries < max_retries:
        try:
            # Fazendo consulta
            response = query_engine_with_rerank.query(query)
            result['answer'] = response.response
            result['contexts'] = get_contexts(response)

            # Avaliando Faithfulness
            eval_result = faith_evaluator.evaluate_response(response=response)
            result['faithfulness'] = eval_result.score
            result['faithfulness_feedback'] = eval_result.feedback

            # Avaliando Relevancy
            eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
            result['relevancy'] = eval_result.score
            result['relevancy_feedback'] = eval_result.feedback

            # Avaliando Correctness
            eval_result = correct_evaluator.evaluate(
                query=query,
                response=response.response,
                reference=ground_truth,
            )
            result['correctness'] = eval_result.score
            result['correctness_feedback'] = eval_result.feedback

            return result

        except Exception as e:
            error_message = str(e)
            if 'rate limit' in error_message.lower():
                retry_after = parse_retry_after(error_message)
                if retry_after:
                    wait_time = float(retry_after)
                else:
                    wait_time = 2 ** retries + random.uniform(0, 1)
                print(f"Rate limit reached. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                raise

    raise Exception("Max retries exceeded")

In [39]:
pairs = [{
            "question": "Qual o nome do IFSP?",
            "answer": "Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.  Referência: Estatuto, Título I, Capítulo I, Art. 1º"
         }
         ,{
            "question": "Quais são os campi do IFSP?",
            "answer": """II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.

Referência: Estatuto, Título I, Capítulo I, Art. 1º, §2º
"""
         }
         ,{
            "question": "Quantos campi o IFSP tem? Informe onde você encontrou essa informação",
            "answer": "O IFSP tem 28 campi. Referência: Estatuto, Capítulo I, Art. 1º, §2º"
         }
        ]

#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

In [41]:
data = []

with ThreadPoolExecutor(max_workers=4) as executor:  # Ajuste o número de workers conforme necessário
    future_to_pair = {executor.submit(evaluate_pair, pair): pair for pair in pairs}
    for future in tqdm(as_completed(future_to_pair), total=len(pairs)):
        try:
            data.append(future.result())
        except Exception as exc:
            print(f'Generated an exception: {exc}')

df = pd.DataFrame(data)
df

  0%|          | 0/3 [00:00<?, ?it/s]

Loading T5Ranker model unicamp-dl/monoptt5-large
No device set
Using device cpu
Device set to `cpu`, setting dtype to `float32`
Using dtype torch.float32
Loading model unicamp-dl/monoptt5-large, this might take a while...
Using device cpu.
Using dtype torch.float32.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5 true token set to ▁Sim
T5 false token set to ▁Não
Returning normalised scores...
Inputs template set to Pergunta: {query} Documento: {text} Relevante:


Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generated an exception: Max retries exceeded
Loading T5Ranker model unicamp-dl/monoptt5-large
No device set
Using device cpu
Device set to `cpu`, setting dtype to `float32`
Using dtype torch.float32
Loading model unicamp-dl/monoptt5-large, this might take a while...
Using device cpu.
Using dtype torch.float32.
T5 true token set to ▁Sim
T5 false token set to ▁Não
Returning normalised scores...
Inputs template set to Pergunta: {query} Documento: {text} Relevante:


Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 3/3 [04:45<00:00, 95.28s/it]

Generated an exception: name 'faith_evaluator' is not defined


""


In [47]:
query = "Ramon Abilio é professor em qual campos do IFSP?"
response = query_engine_with_rerank.query(query)
display_response(response)

Loading T5Ranker model unicamp-dl/monoptt5-large
No device set
Using device cpu
Device set to `cpu`, setting dtype to `float32`
Using dtype torch.float32
Loading model unicamp-dl/monoptt5-large, this might take a while...
Using device cpu.
Using dtype torch.float32.
T5 true token set to ▁Sim
T5 false token set to ▁Não
Returning normalised scores...
Inputs template set to Pergunta: {query} Documento: {text} Relevante:


Scoring...:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** There is no information in the provided context about Ramon Abilio being a professor at the IFSP, so it is not possible to determine which campuses he is a professor at.

# Com rerank veio até aqui

In [37]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [39]:
display_prompt_dict(query_engine_with_rerank.get_prompts())

query = "Qual é o nome do IFSP"

response_rerank = query_engine_with_rerank.query(query)
display_response(response_rerank)

**Prompt Key**: response_synthesizer:text_qa_template **Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template **Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


Retrieving with query id None: Qual é o nome do IFSP
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual é o nome do IFSP
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Qual é o nome do IFSP
Retrieved node with id, entering: node_14
Retrieving with query id node_14: Qual é o nome do IFSP
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Qual é o nome do IFSP
Retrieved node with id, entering: node_3
Retrieving with query id node_3: Qual é o nome do IFSP
Retrieved node with id, entering: node_17
Retrieving with query id node_17: Qual é o nome do IFSP


**`Final Response:`** O nome do IFSP é Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.

In [21]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template **Text:** 

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: response_synthesizer:refine_template **Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [40]:
new_summary_tmpl_str = ("""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:
"""
)

new_refine_tmpl_str = ("""
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Do not add other information, such as 'The refined answer remains the same,' besides the answer and the source.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.

Refined Answer:

"""
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)
new_refine_tmpl_str = PromptTemplate(new_refine_tmpl_str)

query_engine_with_rerank.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl
     , "response_synthesizer:refine_template" : new_refine_tmpl_str}
)

display_prompt_dict(query_engine_with_rerank.get_prompts())


**Prompt Key**: response_synthesizer:text_qa_template **Text:** 


Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:



**Prompt Key**: response_synthesizer:refine_template **Text:** 


The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Do not add other information, such as 'The refined answer remains the same,' besides the answer and the source.
Provide the source of the answer, such as the document, title, chapter, or article when available.
Answer in the same language as the query.

Refined Answer:




In [41]:
query = "Qual o nome do IFSP?"
#query = "Quais são os campi do IFSP? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quantos campi o IFSP tem? Informe onde você encontrou essa informação"
#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

#query = "Sou aluno de graduação e gostaria de saber o que são as transferências especial e ex oficio."
#query = "O que é Estudante Especial?"
# query = "Em qual portaria Ramon Abilio foi designado para compor a Comissão de Avaliação de Atividades Docentes - CAAD?"

response = query_engine_with_rerank.query(query)
display_response(response)

Retrieving with query id None: Qual o nome do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual o nome do IFSP?
Retrieved node with id, entering: node_14
Retrieving with query id node_14: Qual o nome do IFSP?
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Qual o nome do IFSP?
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Qual o nome do IFSP?
Retrieved node with id, entering: node_17
Retrieving with query id node_17: Qual o nome do IFSP?
Retrieved node with id, entering: node_3
Retrieving with query id node_3: Qual o nome do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Qual o nome do IFSP?


**`Final Response:`** O nome do IFSP é Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.

documento: Estatuto, título: TÍTULO I - DA INSTITUIÇÃO, capitulo: CAPÍTULO I DA NATUREZA E DAS FINALIDADES, Art 1º.

In [42]:
pprint(base_nodes[26])

TextNode(
│   id_='node_26',
│   embedding=None,
│   metadata={
│   │   'documento': 'Organização Didática da Educação Básica',
│   │   'titulo': 'TÍTULO III DOS CURSOS DE EDUCAÇÃO BÁSICA PROFISSIONALIZANTE',
│   │   'capitulo': 'CAPÍTULO III DOS CURSOS TÉCNICOS DE NÍVEL MÉDIO NA FORMA INTEGRADA AO ENSINO MÉDIO.'
│   },
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='Art 60 - Serão estruturados em períodos letivos anuais ou semestrais, em turno único ou período integral, com duração de três ou quatro anos, carga horária de acordo com a legislação vigente e distribuição dos dias letivos de acordo com o artigo 17 desta Organização Didática. Parágrafo único. Parte da carga horária poderá ser desenvolvida a distância, quando previstas as atividades não presenciais no Projeto Pedagógico do Curso, respeitando os dispositivos legais vigentes. Art 61 - Os Currículos dos cursos serão organizados em núcleos politécnicos, conforme a seguinte organização: I - Núcleo estruturante comum: relativo às áreas de conhecimento que compõem a formação geral no Ensino Médio, contemplando conteúdos de base científica e cultural basilares para a formação humana integral; II - Núcleo estruturante articulador: relativo às áreas do conhecimento do Ensino Médio e da Educação Profissional, traduzidas em conteúdo de estreita articulação com o curso, por eixo tecnológico, representando elementos expressivos para a integração curricular; III - Núcleo estruturante tecnológico: relativo a conhecimentos da habilitação profissional, de acordo com o campo de conhecimentos do eixo tecnológico, e as regulamentações do exercício da profissão; IV - Prática Profissional: relativo a estágio curricular supervisionado, em situação real de trabalho, optativo ou obrigatório e Projeto Integrador. § 1º. A carga horária total obrigatória dos cursos deverá ser distribuída nos três núcleos estruturantes, de forma que a carga horária referente ao núcleo estruturante articulador poderá compor a carga horária do núcleo estruturante comum ou tecnológico. § 2º. Os componentes curriculares poderão ser organizados em módulos, áreas, projetos, séries anuais, períodos semestrais, ciclos, alternância regular de períodos de estudos e grupos não seriados. Art 62 - As estruturas curriculares deverão seguir as orientações dispostas em capítulo próprio desta Organização Didática. Art 63 - Após a conclusão do curso, inclusive do estágio curricular supervisionado, quando obrigatório, o estudante receberá o diploma de técnico de nível médio. SEÇÃO I - DA VERIFICAÇÃO DO RENDIMENTO ESCOLAR NOS CURSOS TÉCNICOS DE NÍVEL MÉDIO NA FORMA INTEGRADA AO ENSINO MÉDIO Art 64 - Considera-se APROVADO: I - o estudante que obtenha em cada área do conhecimento (Linguagens, Matemática, Ciências da Natureza, Ciências Humanas, Disciplinas Técnicas e Projeto Integrador) média das notas finais igual ou superior a 6,0 (seis), e frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo; II - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo que, após reavaliação obtenha, média das notas finais igual ou superior a 6,0 (seis) em cada área do conhecimento; III - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo, que, após análise do Conselho de Classe Deliberativo, seja considerado aprovado. Art 65 - Considera-se REPROVADO: I - o estudante que obtiver frequência global menor que 75% (setenta e cinco por cento), das aulas dadas e demais atividades no período letivo, independentemente da nota que tiver alcançado; II - o estudante com frequência global mínima de 75% (setenta e cinco por cento) das aulas dadas e demais atividades no período letivo, que, após reavaliação, obtenha média das notas finais menor que 6,0 (seis) em pelo menos uma área do conhecimento

In [43]:
for node in response.source_nodes:
    display_source_node(node)

**Node ID:** node_3<br>**Similarity:** 0.6732763129429461<br>**Text:** SEÇÃO I - DO CONSELHO SUPERIOR Art 11 - O Conselho Superior, de caráter consultivo e deliberativo...<br>

**Node ID:** node_17<br>**Similarity:** 0.6769528431404352<br>**Text:** Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO (IFSP), constituído m...<br>

**Node ID:** node_47<br>**Similarity:** 0.6812914870194932<br>**Text:** Art 1 - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO (IFSP), constituído me...<br>

**Node ID:** node_14<br>**Similarity:** 0.6978638560444979<br>**Text:** Art 44 - O IFSP expedirá e registrará seus diplomas em conformidade com o §3º do art. 2º. da Lei ...<br>

**Node ID:** node_2<br>**Similarity:** 0.6900870971609828<br>**Text:** Art 8º - A organização geral do IFSP compreende: I - ÓRGÃOS SUPERIORES a) Conselho Superior; 1. Ó...<br>

# Avaliando conjunto de perguntas e respostas

In [26]:
pairs = [{
            "question": "Qual o nome do IFSP?",
            "answer": "Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.  Referência: Estatuto, Título I, Capítulo I, Art. 1º"
         }
         ,{
            "question": "Quais são os campi do IFSP?",
            "answer": """II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.

Referência: Estatuto, Título I, Capítulo I, Art. 1º, §2º
"""
         }
         ,{
            "question": "Quantos campi o IFSP tem? Informe onde você encontrou essa informação",
            "answer": "O IFSP tem 28 campi. Referência: Estatuto, Capítulo I, Art. 1º, §2º"
         }
        ]

#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

In [45]:
def get_contexts(response):
    contexts = []
    for node in response.source_nodes:
        metadata = ', '.join(node.metadata.values())
        context  = node.get_content()
        contexts.append(f"Fonte: {metadata} # Contextos: {context}")
    return contexts

In [54]:
faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []

for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    response           = query_engine_with_rerank.query(query)
    result['answer']   = response.response
    result['contexts'] = get_contexts(response)


    # Avaliando Faithfulness
    eval_result                     = faith_evaluator.evaluate_response(response=response)
    result['faithfulness']          = eval_result.score
    result['faithfulness_feedback'] = eval_result.feedback

    # Avaliando Relevancy
    eval_result                  = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy']          = eval_result.score
    result['relevancy_feedback'] = eval_result.feedback

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness']          = eval_result.score
    result['correctness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving with query id None: Qual o nome do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual o nome do IFSP?
Retrieved node with id, entering: node_14
Retrieving with query id node_14: Qual o nome do IFSP?
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Qual o nome do IFSP?
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Qual o nome do IFSP?
Retrieved node with id, entering: node_17
Retrieving with query id node_17: Qual o nome do IFSP?
Retrieved node with id, entering: node_3
Retrieving with query id node_3: Qual o nome do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Qual o nome do IFSP?


 33%|███▎      | 1/3 [02:05<04:10, 125.04s/it]

Retrieving with query id None: Quais são os campi do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quais são os campi do IFSP?
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Quais são os campi do IFSP?
Retrieved node with id, entering: node_15
Retrieving with query id node_15: Quais são os campi do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Quais são os campi do IFSP?
Retrieved node with id, entering: node_53
Retrieving with query id node_53: Quais são os campi do IFSP?


 67%|██████▋   | 2/3 [06:56<03:42, 222.86s/it]

Retrieving with query id None: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_75
Retrieving with query id node_75: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_53
Retrieving with query id node_53: Quantos campi o IFSP tem? Informe onde você encontrou essa informação


100%|██████████| 3/3 [13:10<00:00, 263.56s/it]


,question,ground_truth,answer,contexts,faithfulness,faithfulness_feedback,relevancy,relevancy_feedback,correctness,correctness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",O nome do IFSP é o Instituto Federal de Educaç...,"[Fonte: Estatuto, TÍTULO II - DA GESTÃO, CAPÍT...",1.0,YES,1.0,YES,5.0,The generated answer is fully correct and rele...
1,Quais são os campi do IFSP?,"II. Campus São Paulo, situado na Rua Pedro Vic...","Os campi do IFSP são:\n\nI - Reitoria, no ende...","[Fonte: Estatuto, TÍTULO VI - DO PATRIMÔNIO. #...",1.0,YES,1.0,YES,5.0,The generated answer is fully correct and rele...
2,Quantos campi o IFSP tem? Informe onde você en...,"O IFSP tem 28 campi. Referência: Estatuto, Cap...",A resposta é: O IFSP tem 31 campi.\n\nFonte: D...,[Fonte: Organização Didática dos Cursos de Gra...,1.0,YES,1.0,"The existing answer is YES, and the new contex...",3.0,The generated answer is relevant to the user q...


In [53]:
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def parse_retry_after(error_message):
    try:
        error_data = json.loads(error_message)
        return error_data['error']['message'].split('Please try again in ')[1].split('s.')[0]
    except Exception as e:
        print(f"Failed to parse retry-after duration: {e}")
        return None

def evaluate_pair(pair, max_retries=2):
    result = {}
    query = pair['question']
    ground_truth = pair['answer']

    result['question'] = query
    result['ground_truth'] = ground_truth

    retries = 0
    while retries < max_retries:
        try:
            # Fazendo consulta
            response = query_engine_with_rerank.query(query)
            result['answer'] = response.response
            result['contexts'] = get_contexts(response)

            # Avaliando Faithfulness
            eval_result = faith_evaluator.evaluate_response(response=response)
            result['faithfulness'] = eval_result.score
            result['faithfulness_feedback'] = eval_result.feedback

            # Avaliando Relevancy
            eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
            result['relevancy'] = eval_result.score
            result['relevancy_feedback'] = eval_result.feedback

            # Avaliando Correctness
            eval_result = correct_evaluator.evaluate(
                query=query,
                response=response.response,
                reference=ground_truth,
            )
            result['correctness'] = eval_result.score
            result['correctness_feedback'] = eval_result.feedback

            return result

        except Exception as e:
            error_message = str(e)
            if 'rate limit' in error_message.lower():
                retry_after = parse_retry_after(error_message)
                if retry_after:
                    wait_time = float(retry_after)
                else:
                    wait_time = 2 ** retries + random.uniform(0, 1)
                print(f"Rate limit reached. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                raise

    raise Exception("Max retries exceeded")

data = []

with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust the number of workers as needed
    future_to_pair = {executor.submit(evaluate_pair, pair): pair for pair in pairs}
    for future in tqdm(as_completed(future_to_pair), total=len(pairs)):
        try:
            data.append(future.result())
        except Exception as exc:
            print(f'Generated an exception: {exc}')

df = pd.DataFrame(data)
df

Retrieving with query id None: Qual o nome do IFSP?
Retrieving with query id None: Quais são os campi do IFSP?
Retrieving with query id None: Quantos campi o IFSP tem? Informe onde você encontrou essa informação


  0%|          | 0/3 [00:00<?, ?it/s]

Retrieved node with id, entering: node_0
Retrieving with query id node_0: Qual o nome do IFSP?
Retrieved node with id, entering: node_14
Retrieving with query id node_14: Qual o nome do IFSP?
Retrieved node with id, entering: node_2
Retrieving with query id node_2: Qual o nome do IFSP?
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Qual o nome do IFSP?
Retrieved node with id, entering: node_17
Retrieving with query id node_17: Qual o nome do IFSP?
Retrieved node with id, entering: node_3
Retrieving with query id node_3: Qual o nome do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Qual o nome do IFSP?
Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: node_75
Retrieving with query id node_75: Quantos campi o IFSP tem? Informe onde você encontrou essa informação
Retrieved node with id, entering: nod

Failed to parse retry-after duration: Expecting value: line 1 column 1 (char 0)
Rate limit reached. Retrying in 1.07 seconds...
Retrieving with query id None: Quais são os campi do IFSP?


Retrieved node with id, entering: node_0
Retrieving with query id node_0: Quais são os campi do IFSP?
Retrieved node with id, entering: node_47
Retrieving with query id node_47: Quais são os campi do IFSP?
Retrieved node with id, entering: node_15
Retrieving with query id node_15: Quais são os campi do IFSP?
Retrieved node with id, entering: node_10
Retrieving with query id node_10: Quais são os campi do IFSP?
Retrieved node with id, entering: node_53
Retrieving with query id node_53: Quais são os campi do IFSP?


...

Failed to parse retry-after duration: Expecting value: line 1 column 1 (char 0)
Rate limit reached. Retrying in 2.89 seconds...


 67%|██████▋   | 2/3 [00:22<00:10, 10.81s/it]

Generated an exception: Max retries exceeded


100%|██████████| 3/3 [03:28<00:00, 69.43s/it]


,question,ground_truth,answer,contexts,faithfulness,faithfulness_feedback,relevancy,relevancy_feedback,correctness,correctness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",O nome do IFSP é o Instituto Federal de Educaç...,"[Fonte: Estatuto, TÍTULO II - DA GESTÃO, CAPÍT...",1.0,YES,1.0,YES,5.0,The generated answer is fully correct and rele...


In [ ]:
all_no